In [1]:
# ЯЧЕЙКА 1: Импорты и базовые настройки
import os
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm import tqdm

RMT_INFER_DIR = "/mnt/10tb/home/patsukevichan/borZoi/borzoi/examples/rmt_infer/"
BORZOI_BEDGRAPH_DIR = "/mnt/10tb/home/patsukevichan/borZoi/borzoi/examples/predicted_expression_by_chromosomes/"

print("Libraries imported. Paths set.")


Libraries imported. Paths set.


In [2]:
# ЯЧЕЙКА 2: Загрузка базовых parquet-файлов (labels, transcripts, RMT-предикты)

labels_parquet = os.path.join(RMT_INFER_DIR, "labels.parquet")
labels_transcripts_parquet = os.path.join(RMT_INFER_DIR, "labels_transcripts.parquet")
preds_2seg_parquet = os.path.join(RMT_INFER_DIR, "preds_transcripts_2seg.parquet")
preds_4seg_parquet = os.path.join(RMT_INFER_DIR, "preds_transcripts_4seg.parquet")
preds_8seg_parquet = os.path.join(RMT_INFER_DIR, "preds_transcripts_8seg.parquet")

labels_df = pd.read_parquet(labels_parquet)
labels_transcripts_df = pd.read_parquet(labels_transcripts_parquet)
preds_2seg_df = pd.read_parquet(preds_2seg_parquet)
preds_4seg_df = pd.read_parquet(preds_4seg_parquet)
preds_8seg_df = pd.read_parquet(preds_8seg_parquet)

print("labels_df:")
display(labels_df.head())
print("labels_df[preds].min/max/mean:",
      labels_df["preds"].min(), 
      labels_df["preds"].max(), 
      labels_df["preds"].mean())

print("\nlabels_transcripts_df:")
display(labels_transcripts_df.head())
print("labels_transcripts_df[preds].min/max/mean:",
      labels_transcripts_df["preds"].min(),
      labels_transcripts_df["preds"].max(),
      labels_transcripts_df["preds"].mean())

print("\npreds_2seg_df:")
display(preds_2seg_df.head())
print("preds_2seg_df[preds].min/max/mean:",
      preds_2seg_df["preds"].min(),
      preds_2seg_df["preds"].max(),
      preds_2seg_df["preds"].mean())

print("\npreds_4seg_df:")
display(preds_4seg_df.head())
print("preds_4seg_df[preds].min/max/mean:",
      preds_4seg_df["preds"].min(),
      preds_4seg_df["preds"].max(),
      preds_4seg_df["preds"].mean())

print("\npreds_8seg_df:")
display(preds_8seg_df.head())
print("preds_8seg_df[preds].min/max/mean:",
      preds_8seg_df["preds"].min(),
      preds_8seg_df["preds"].max(),
      preds_8seg_df["preds"].mean())


labels_df:


,chrom_pred,preds,chrom,start,end,start_pred,end_pred,Transcript stable ID,description_id
0,chr10,-4.60517,chr10,72893738,72893859,72893735,72893739,ENST00000334011,ENCFF123KIW
1,chr10,-4.60517,chr10,72893738,72893859,72893739,72893743,ENST00000334011,ENCFF123KIW
2,chr10,-4.60517,chr10,72893738,72893859,72893743,72893749,ENST00000334011,ENCFF123KIW
3,chr10,-4.60517,chr10,72893738,72893859,72893749,72893752,ENST00000334011,ENCFF123KIW
4,chr10,-4.60517,chr10,72893738,72893859,72893752,72893758,ENST00000334011,ENCFF123KIW


labels_df[preds].min/max/mean:
 -4.6051702 13.022035 1.0628820826820662

labels_transcripts_df:


,Transcript stable ID,description_id,preds
0,ENST00000001008,ENCFF123KIW,4.037295
1,ENST00000002596,ENCFF123KIWr,4.281860
2,ENST00000002829,ENCFF123KIW,5.388129
3,ENST00000009180,ENCFF123KIW,7.723541
4,ENST00000009589,ENCFF123KIWr,8.866660


labels_transcripts_df[preds].min/max/mean:
 -4.605170200000001 11.856355566803279 -0.4990853600718094

preds_2seg_df:


...

preds_2seg_df[preds].min/max/mean:
 -3.8326391907692305 6.641538864634146 -1.7986488347158898

preds_4seg_df:


,Transcript stable ID,description_id,preds
0,ENST00000001008,ENCFF123KIW,2.727709
1,ENST00000002596,ENCFF123KIWr,0.732307
2,ENST00000002829,ENCFF123KIW,2.426938
3,ENST00000009180,ENCFF123KIW,4.060136
4,ENST00000009589,ENCFF123KIWr,4.862711


preds_4seg_df[preds].min/max/mean:
 -4.2842083040022745 7.001022550393701 -0.715264122164497

preds_8seg_df:


,Transcript stable ID,description_id,preds
0,ENST00000001008,ENCFF123KIW,3.736615
1,ENST00000002596,ENCFF123KIWr,1.674160
2,ENST00000002829,ENCFF123KIW,0.736134
3,ENST00000009180,ENCFF123KIW,4.146430
4,ENST00000009589,ENCFF123KIWr,5.075033


preds_8seg_df[preds].min/max/mean:
 -4.180466452112676 6.389540544094489 -0.6615956298938233


In [3]:
# ЯЧЕЙКА 3: Формируем "очищенный" DataFrame labels_clear_df
# Убираем лишний столбец preds, оставляем [chrom, start, end, Transcript stable ID, description_id]

labels_clear_df = labels_df[["chrom", "start", "end", "Transcript stable ID", "description_id"]].copy()
labels_clear_df.drop_duplicates(subset=["Transcript stable ID"], inplace=True)

print("labels_clear_df (очищенная):")
display(labels_clear_df.head())
print("Shape:", labels_clear_df.shape)


labels_clear_df (очищенная):


,chrom,start,end,Transcript stable ID,description_id
0,chr10,72893738,72893859,ENST00000334011,ENCFF123KIW
379,chr10,73071294,73071357,ENST00000431293,ENCFF123KIW
425,chr10,73110454,73110567,ENST00000357321,ENCFF123KIW
772,chr10,73168118,73168386,ENST00000242505,ENCFF123KIW
1660,chr10,73247359,73247383,ENST00000440197,ENCFF123KIW


Shape: (4419, 5)


In [4]:
# ЯЧЕЙКА 4: Сопоставление description_id -> имя bedGraph-файла (Borzoi)
# Предположим, что вы уже знаете соответствие.

experiment_mapping = {
    "ENCFF123KIW":  "borzoi_rnaseq_fold4_ENCSR892LBU+_fw.bedGraph",
    "ENCFF123KIWr": "borzoi_rnaseq_fold4_ENCSR892LBU-_fw.bedGraph",
    "ENCFF784MDF":  "borzoi_rnaseq_fold4_ENCSR357BYU+_fw.bedGraph",
    "ENCFF784MDFr": "borzoi_rnaseq_fold4_ENCSR357BYU-_fw.bedGraph",
    "ENCFF236XOK":  "borzoi_rnaseq_fold4_ENCSR763OMY+_fw.bedGraph",
    "ENCFF236XOKr": "borzoi_rnaseq_fold4_ENCSR763OMY-_fw.bedGraph",
    "ENCFF781TTC":  "borzoi_rnaseq_fold4_ENCSR071DYD+_fw.bedGraph",
    "ENCFF781TTCr": "borzoi_rnaseq_fold4_ENCSR071DYD-_fw.bedGraph",
    "ENCFF242BWW":  "borzoi_rnaseq_fold4_ENCSR045GTF+_fw.bedGraph",
    "ENCFF242BWWr": "borzoi_rnaseq_fold4_ENCSR045GTF-_fw.bedGraph"
}

print("experiment_mapping (description_id -> bedGraph file):")
for k,v in experiment_mapping.items():
    print(k, "->", v)


Словарь сопоставлений для description_id:
ENCFF123KIW: borzoi_rnaseq_fold4_ENCSR892LBU+_fw.bedGraph
ENCFF123KIWr: borzoi_rnaseq_fold4_ENCSR892LBU-_fw.bedGraph
ENCFF784MDF: borzoi_rnaseq_fold4_ENCSR357BYU+_fw.bedGraph
ENCFF784MDFr: borzoi_rnaseq_fold4_ENCSR357BYU-_fw.bedGraph
ENCFF236XOK: borzoi_rnaseq_fold4_ENCSR763OMY+_fw.bedGraph
ENCFF236XOKr: borzoi_rnaseq_fold4_ENCSR763OMY-_fw.bedGraph
ENCFF781TTC: borzoi_rnaseq_fold4_ENCSR071DYD+_fw.bedGraph
ENCFF781TTCr: borzoi_rnaseq_fold4_ENCSR071DYD-_fw.bedGraph
ENCFF242BWW: borzoi_rnaseq_fold4_ENCSR045GTF+_fw.bedGraph
ENCFF242BWWr: borzoi_rnaseq_fold4_ENCSR045GTF-_fw.bedGraph


In [5]:
# ЯЧЕЙКА 5: Загрузка bedGraph-файлов Borzoi (fw) в оперативную память
from tqdm import tqdm

bedgraph_files = list(set(experiment_mapping.values()))
bedgraph_data = {}

for fname in tqdm(bedgraph_files, desc="Loading bedGraph files"):
    full_path = os.path.join(BORZOI_BEDGRAPH_DIR, fname)
    if not os.path.exists(full_path):
        print("Файл не найден:", fname)
        continue
    df_tmp = pd.read_csv(full_path, sep="\t", header=None, names=["chrom","start","end","value"])
    bedgraph_data[fname] = df_tmp
    
    print(f"\nЗагружен {fname}, shape={df_tmp.shape}")
    display(df_tmp.head(3))
    print("value stats: min=%.3f max=%.3f mean=%.3f" % 
          (df_tmp["value"].min(), df_tmp["value"].max(), df_tmp["value"].mean()))


Обработка файлов:   0%|                                                                                                                                          | 0/10 [00:00<?, ?it/s]


Загружен файл borzoi_rnaseq_fold4_ENCSR357BYU+_fw.bedGraph, shape=(10799537, 4)


,chrom,start,end,value
0,chr10,72890684,72890716,0.538787
1,chr10,72890716,72890748,0.216714
2,chr10,72890748,72890780,0.051275


Обработка файлов:  10%|█████████████                                                                                                                     | 1/10 [00:03<00:32,  3.59s/it]

value: min=0.000 max=4004.835 mean=1.180

Загружен файл borzoi_rnaseq_fold4_ENCSR892LBU-_fw.bedGraph, shape=(10799537, 4)


...

Обработка файлов:  90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████             | 9/10 [00:58<00:06,  6.40s/it]

value: min=0.000 max=4382.543 mean=1.627

Загружен файл borzoi_rnaseq_fold4_ENCSR763OMY+_fw.bedGraph, shape=(10799537, 4)


,chrom,start,end,value
0,chr10,72890684,72890716,0.141283
1,chr10,72890716,72890748,0.042791
2,chr10,72890748,72890780,0.006480


Обработка файлов: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:05<00:00,  6.54s/it]

value: min=0.000 max=9742.334 mean=2.038


In [6]:
# ЯЧЕЙКА 6: Логарифмирование coverage 

PSEUDOCOUNT = 0.01

bedgraph_data_log = {}
for fname, df in bedgraph_data.items():
    df_log = df.copy()
    # логарифмируем: ln(value + 0.01)
    df_log["value"] = np.log(df_log["value"] + PSEUDOCOUNT)
    bedgraph_data_log[fname] = df_log
    
    print(f"\nЛогарифмирован файл {fname}")
    print("value(min/max/mean) =", 
          df_log["value"].min(), 
          df_log["value"].max(), 
          df_log["value"].mean())



Логарифмирован файл borzoi_rnaseq_fold4_ENCSR357BYU+_fw.bedGraph
value(min/max/mean) = -4.604050246033248 8.29526008641543 -2.782012479944866

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR892LBU-_fw.bedGraph
value(min/max/mean) = -4.603217062228037 8.263734575941122 -2.3667830958282927

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR045GTF-_fw.bedGraph
value(min/max/mean) = -4.603812122672792 8.33599800939897 -2.4426745504733556

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR357BYU-_fw.bedGraph
value(min/max/mean) = -4.603574056001573 8.070751273337727 -2.7398935592542215

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR763OMY-_fw.bedGraph
value(min/max/mean) = -4.604979469312392 8.779723568889084 -3.0078493281054324

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR892LBU+_fw.bedGraph
value(min/max/mean) = -4.604097877510423 8.636466477209188 -2.470956381734402

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR045GTF+_fw.bedGraph
value(min/max/mean) = -4.604383714027222 8.765460701302153 -2.523270

In [7]:
# ЯЧЕЙКА 7: Подсчёт transcript-level покрытий Борзого, с учётом хромосомы [start..end]

preds_borzoi = []

for _, row in tqdm(labels_transcripts_df.iterrows(), total=labels_transcripts_df.shape[0], desc="Borzoi coverage"):
    t_id   = row["Transcript stable ID"]
    did    = row["description_id"]  
    ideal  = row["preds"]  # лог "идеальных меток" (RMT vs ...)

    # Ищем координаты [chrom, start, end] в labels_clear_df
    sub = labels_clear_df[labels_clear_df["Transcript stable ID"] == t_id]
    if sub.empty:
        preds_borzoi.append([t_id, did, np.nan, ideal])
        continue
    
    # Возьмём первую строку
    chrom  = sub.iloc[0]["chrom"]
    start_ = sub.iloc[0]["start"]
    end_   = sub.iloc[0]["end"]
    
    # Находим имя bedGraph
    bedfname = experiment_mapping.get(did, None)
    if bedfname is None:
        preds_borzoi.append([t_id, did, np.nan, ideal])
        continue
    
    # Достаём лог-файл
    df_cov = bedgraph_data_log.get(bedfname, None)
    if df_cov is None:
        preds_borzoi.append([t_id, did, np.nan, ideal])
        continue
    
    # Фильтруем: та же хромосома и [start..end]
    region = df_cov[
        (df_cov["chrom"] == chrom) &
        (df_cov["start"] >= start_) &
        (df_cov["end"]   <= end_)
    ]
    
    if region.empty:
        # если пусто: берём 0.0 (что в лог.шкале ~ ln(0.01)= -4.6)
        preds_borzoi.append([t_id, did, 0.0, ideal])
    else:
        mean_logcov = region["value"].mean()
        preds_borzoi.append([t_id, did, mean_logcov, ideal])

preds_borzoi_df = pd.DataFrame(preds_borzoi, columns=["Transcript stable ID","description_id","pred_borzoi","pred_ideal"])
print("preds_borzoi_df shape:", preds_borzoi_df.shape)
display(preds_borzoi_df.head(10))


Borzoi coverage: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22095/22095 [3:20:05<00:00,  1.84it/s]


In [9]:
# ЯЧЕЙКА 8: Объединяем c RMT (2/4/8seg)

df_merged = preds_borzoi_df.copy()

# 2seg
temp = preds_2seg_df[["Transcript stable ID","description_id","preds"]].rename(columns={"preds":"pred_rmt_2"})
df_merged = pd.merge(df_merged, temp, on=["Transcript stable ID","description_id"], how="left")

# 4seg
temp = preds_4seg_df[["Transcript stable ID","description_id","preds"]].rename(columns={"preds":"pred_rmt_4"})
df_merged = pd.merge(df_merged, temp, on=["Transcript stable ID","description_id"], how="left")

# 8seg
temp = preds_8seg_df[["Transcript stable ID","description_id","preds"]].rename(columns={"preds":"pred_rmt_8"})
df_merged = pd.merge(df_merged, temp, on=["Transcript stable ID","description_id"], how="left")

print("df_merged:", df_merged.shape)
display(df_merged.head())

# Статистика и корреляция (транскрипт-уровень)
for col in ["pred_borzoi","pred_ideal","pred_rmt_2","pred_rmt_4","pred_rmt_8"]:
    if col in df_merged.columns:
        print(f"{col} min={df_merged[col].min()} max={df_merged[col].max()} mean={df_merged[col].mean()}")

corr_cols = ["pred_borzoi","pred_ideal","pred_rmt_2","pred_rmt_4","pred_rmt_8"]
df_corr = df_merged[corr_cols].corr(method="pearson")
print("\nTranscript-level корреляции (Pearson):")
display(df_corr)


df_merged final shape: (22095, 7)


,Transcript stable ID,description_id,pred_borzoi,pred_ideal,pred_rmt_2,pred_rmt_4,pred_rmt_8
0,ENST00000001008,ENCFF123KIW,4.712871,4.037295,0.570057,2.727709,3.736615
1,ENST00000002596,ENCFF123KIWr,1.099219,4.281860,-2.881656,0.732307,1.674160
2,ENST00000002829,ENCFF123KIW,3.475321,5.388129,1.192969,2.426938,0.736134
3,ENST00000009180,ENCFF123KIW,6.688652,7.723541,2.320119,4.060136,4.146430
4,ENST00000009589,ENCFF123KIWr,5.853962,8.866660,4.619201,4.862711,5.075033


pred_borzoi min=-4.578855155245346 max=8.14484626177008 mean=0.19121216321998807
pred_ideal min=-4.605170200000001 max=11.856355566803279 mean=-0.4990853600718094
pred_rmt_2 min=-3.8326391907692305 max=6.641538864634146 mean=-1.7986488347158898
pred_rmt_4 min=-4.2842083040022745 max=7.001022550393701 mean=-0.715264122164497
pred_rmt_8 min=-4.180466452112676 max=6.389540544094489 mean=-0.6615956298938233

Корреляции (Pearson):


,pred_borzoi,pred_ideal,pred_rmt_2,pred_rmt_4,pred_rmt_8
pred_borzoi,1.000000,0.784860,0.657547,0.779853,0.782714
pred_ideal,0.784860,1.000000,0.666830,0.781221,0.778404
pred_rmt_2,0.657547,0.666830,1.000000,0.818327,0.773918
pred_rmt_4,0.779853,0.781221,0.818327,1.000000,0.960751
pred_rmt_8,0.782714,0.778404,0.773918,0.960751,1.000000


In [17]:
# ЯЧЕЙКА 9: Загрузка t_g_df -> хранит сопоставление transcript->gene

t_g_file = os.path.join(RMT_INFER_DIR, "t_g.tsv")
t_g_df = pd.read_csv(t_g_file, sep="\t")

print("Первые 5 строк t_g_df:")
display(t_g_df.head())

print("\nИнформация о t_g_df:")
t_g_df.info()


Первые 5 строк t_g_df:


,Unnamed: 0,chrom,name,strand,strand.1,split,Transcript stable ID,start,end
0,29191,chr3,ENST00000002829,1,ENSG00000001617,valid,ENST00000002829,50140727,50189075
1,29192,chr12,ENST00000358495,-1,ENSG00000002016,valid,ENST00000358495,911735,964290
2,29193,chr4,ENST00000226299,1,ENSG00000002549,valid,ENST00000226299,17562601,17607970
3,29194,chr4,ENST00000002596,-1,ENSG00000002587,valid,ENST00000002596,11393149,11443490
4,29195,chr2,ENST00000264276,-1,ENSG00000003393,valid,ENST00000264276,201700266,201795529



Информация о t_g_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4419 entries, 0 to 4418
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            4419 non-null   int64 
 1   chrom                 4419 non-null   object
 2   name                  4419 non-null   object
 3   strand                4419 non-null   int64 
 4   strand.1              4419 non-null   object
 5   split                 4419 non-null   object
 6   Transcript stable ID  4419 non-null   object
 7   start                 4419 non-null   int64 
 8   end                   4419 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 310.8+ KB


In [39]:
# =================== ЯЧЕЙКА 9 ===================
# Загрузка TPM-таблиц ENCODE (для каждого эксперимента) и сохранение в словарь.
# (Предполагается, что df_merged у нас уже есть из предыдущих ячеек)

import os
import numpy as np
import pandas as pd

RMT_INFER_DIR = "/mnt/10tb/home/patsukevichan/borZoi/borzoi/examples/rmt_infer/"

#https://www.encodeproject.org/experiments/ENCSR892LBU/
#https://www.encodeproject.org/experiments/ENCSR357BYU/
#https://www.encodeproject.org/experiments/ENCSR763OMY/
#https://www.encodeproject.org/experiments/ENCSR071DYD/
#https://www.encodeproject.org/experiments/ENCSR045GTF/

encode_gene_tpm_files = {
    "ENCSR071DYD": "ENCFF258PUQ ENCSR071DYD.tsv",
    "ENCSR763OMY": "ENCFF993FDD ENCSR763OMY.tsv",
    "ENCSR357BYU": "ENCFF296APA ENCSR357BYU.tsv",
    "ENCSR892LBU": "ENCFF653OLW ENCSR892LBU.tsv",
    "ENCSR045GTF": "ENCFF769IPE ENCSR045GTF.tsv"
}

def parse_ensg_to_num(ensg_str: str):
    """
    Пример: "ENSG00000001617.11" -> "ENSG00000001617" -> "00000001617" -> 1617 (int).
    Если не удаётся, возвращаем np.nan.
    """
    if pd.isna(ensg_str):
        return np.nan
    # Убираем версию:
    base = ensg_str.split('.', 1)[0]  # "ENSG00000001617.11" => "ENSG00000001617"
    # Убираем префикс ENSG:
    if base.startswith("ENSG"):
        base = base[4:]  # "00000001617"
    # Срезаем ведущие нули:
    base = base.lstrip('0')  # "1617"
    if base == '':
        return np.nan
    try:
        return int(base)
    except ValueError:
        return np.nan

def parse_any_gene_id(gid_str: str) -> float:
    """
    Пытаемся считать gid_str как int. 
    Если не вышло (ValueError), интерпретируем как ENSG (через parse_ensg_to_num).
    Если и это не вышло, возвращаем NaN.
    """
    if pd.isna(gid_str):
        return np.nan

    # 1) Сначала пробуем обычный int
    try:
        return int(gid_str)
    except ValueError:
        pass

    # 2) Пробуем ENSG->число
    val = parse_ensg_to_num(gid_str)
    return val

encode_gene_tpm_data = {}

for experiment_id, fname in encode_gene_tpm_files.items():
    fpath = os.path.join(RMT_INFER_DIR, fname)
    if not os.path.exists(fpath):
        print(f"[WARN] Файл TPM для эксперимента {experiment_id} не найден: {fname}")
        continue

    df_tpm = pd.read_csv(fpath, sep="\t")
    # Предположим, что нас интересуют столбцы: 'gene_id', 'transcript_id(s)', 'TPM'
    if not {"gene_id","transcript_id(s)","TPM"}.issubset(df_tpm.columns):
        print(f"[WARN] Отсутствуют нужные колонки в {fname}. Пропуск.")
        continue

    # Приводим gene_id к числу (int или float с NaN)
    df_tpm["gene_id"] = df_tpm["gene_id"].apply(parse_any_gene_id)
    # Важно: если хотите, можно привести к Int64, 
    # но допустим пока оставим float, чтобы NaN мог быть сохранён.

    print(f"\n[OK] Загрузили файл: {fname} (experiment_id={experiment_id}), shape={df_tpm.shape}")
    display(df_tpm.head(3))

    encode_gene_tpm_data[experiment_id] = df_tpm



[OK] Загрузили файл: ENCFF258PUQ ENCSR071DYD.tsv (experiment_id=ENCSR071DYD), shape=(59526, 17)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904.0,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954.0,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956.0,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



[OK] Загрузили файл: ENCFF993FDD ENCSR763OMY.tsv (experiment_id=ENCSR763OMY), shape=(59526, 17)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904.0,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954.0,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956.0,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



[OK] Загрузили файл: ENCFF296APA ENCSR357BYU.tsv (experiment_id=ENCSR357BYU), shape=(59526, 17)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904.0,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954.0,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956.0,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



[OK] Загрузили файл: ENCFF653OLW ENCSR892LBU.tsv (experiment_id=ENCSR892LBU), shape=(59526, 17)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904.0,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954.0,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956.0,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



[OK] Загрузили файл: ENCFF769IPE ENCSR045GTF.tsv (experiment_id=ENCSR045GTF), shape=(59526, 17)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904.0,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954.0,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956.0,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
# =================== ЯЧЕЙКА 10 ===================
# Подготавливаем функции strip_enst_version(...) и parse_transcript_ids(...)

def strip_enst_version(enst_str):
    """
    "ENST00000001008.2" -> "ENST00000001008"
    Если нет точки, возвращаем строку без изменений.
    """
    if '.' in enst_str:
        return enst_str.split('.', 1)[0]
    return enst_str

def parse_transcript_ids(text):
    """
    Разбиваем строку типа: "ENST00000001008.2,ENST00000002829.3" на список
    ["ENST00000001008","ENST00000002829"] (обрезая версии).
    """
    if pd.isna(text):
        return []
    raw_ids = text.split(',')
    cleaned_ids = []
    for rid in raw_ids:
        rid_stripped = rid.strip()
        rid_stripped = strip_enst_version(rid_stripped)
        cleaned_ids.append(rid_stripped)
    return cleaned_ids


In [50]:
# =================== ЯЧЕЙКА 11 ===================
# Собираем единый encode_union_df

encode_union_list = []

for experiment_id, df_tpm in encode_gene_tpm_data.items():
    df_tmp = df_tpm.copy()

    # Создаём список транскриптов (без версий)
    df_tmp["list_of_transcripts"] = df_tmp["transcript_id(s)"].apply(parse_transcript_ids)
    
    # explode
    df_exploded = df_tmp.explode("list_of_transcripts", ignore_index=True)
    
    # Переименуем столбец
    df_exploded.rename(columns={"list_of_transcripts": "Transcript stable ID"}, inplace=True)
    
    # Добавим "experiment"
    df_exploded["experiment"] = experiment_id
    
    # Оставим нужные столбцы
    df_exploded = df_exploded[["Transcript stable ID","experiment","gene_id","TPM"]]
    
    encode_union_list.append(df_exploded)

encode_union_df = pd.concat(encode_union_list, ignore_index=True)

print(f"encode_union_df shape = {encode_union_df.shape}")
display(encode_union_df.head(5))
encode_union_df[encode_union_df["Transcript stable ID"].str.startswith("ENST")].head(10)


encode_union_df shape = (1037535, 4)


,Transcript stable ID,experiment,gene_id,TPM
0,10904,ENCSR071DYD,10904.0,0.0
1,12954,ENCSR071DYD,12954.0,0.0
2,12956,ENCSR071DYD,12956.0,0.0
3,12958,ENCSR071DYD,12958.0,0.0
4,12960,ENCSR071DYD,12960.0,0.0


,Transcript stable ID,experiment,gene_id,TPM
649,ENST00000373020,ENCSR071DYD,3.0,7.78
650,ENST00000494424,ENCSR071DYD,3.0,7.78
651,ENST00000496771,ENCSR071DYD,3.0,7.78
652,ENST00000612152,ENCSR071DYD,3.0,7.78
653,ENST00000614008,ENCSR071DYD,3.0,7.78
654,ENST00000373031,ENCSR071DYD,5.0,0.00
655,ENST00000485971,ENCSR071DYD,5.0,0.00
656,ENST00000371582,ENCSR071DYD,419.0,5.17
657,ENST00000371584,ENCSR071DYD,419.0,5.17
658,ENST00000371588,ENCSR071DYD,419.0,5.17


In [53]:
# =================== ЯЧЕЙКА 12 ===================

# df_merged: ["Transcript stable ID","description_id","pred_borzoi",
#             "pred_ideal","pred_rmt_2","pred_rmt_4","pred_rmt_8","experiment"]

# 1) Убираем версию у Transcript stable ID
df_merged["Transcript stable ID"] = df_merged["Transcript stable ID"].apply(strip_enst_version)

# 2) Мерджим
df_final = pd.merge(
    df_merged,
    encode_union_df,
    on=["Transcript stable ID", "experiment"],
    how="left"
)

print("df_final shape =", df_final.shape)
display(df_final.head(5))

# Отладка: характеристики TPM до логарифмирования
print("\nTPM statistics BEFORE log transformation:")
print(f"Min: {df_final['TPM'].min()}")
print(f"Max: {df_final['TPM'].max()}")
print(f"Mean: {df_final['TPM'].mean()}")
print(f"Median: {df_final['TPM'].median()}")
print(f"Std: {df_final['TPM'].std()}")
print(f"Count of zeros: {(df_final['TPM'] == 0).sum()}")

# 2.1) Логарифмируем TPM значения перед расчетом корреляции.
PSEUDOCOUNT = 0.01
df_final["TPM"] = np.log(df_final["TPM"] + PSEUDOCOUNT)

# Отладка: характеристики TPM после логарифмирования
print("\nTPM statistics AFTER log transformation:")
print(f"Min: {df_final['TPM'].min()}")
print(f"Max: {df_final['TPM'].max()}")
print(f"Mean: {df_final['TPM'].mean()}")
print(f"Median: {df_final['TPM'].median()}")
print(f"Std: {df_final['TPM'].std()}")

# 3) Задаем порядок столбцов так, чтобы pred_ideal был предпоследним, а TPM последним
analysis_cols = ["pred_borzoi", "pred_rmt_2", "pred_rmt_4", "pred_rmt_8", "pred_ideal", "TPM"]

# Рассчитываем transcript-level корреляцию
df_for_corr_all = df_final[analysis_cols].dropna()
print(f"\nВсего строк в df_final: {len(df_final)}, после dropna={len(df_for_corr_all)}")

if len(df_for_corr_all) < 5:
    print("Слишком мало данных для transcript-level корреляции.")
else:
    cor_all = df_for_corr_all.corr(method="pearson")
    print("\n[Transcript-level] Общая корреляция:")
    display(cor_all)

# 4) Gene-level корреляция (группируем по gene_id)
df_gene_all = df_final.dropna(subset=["gene_id"])[["gene_id"] + analysis_cols]
df_gene_all = df_gene_all.groupby("gene_id", dropna=False).mean(numeric_only=True).reset_index()

df_gene_for_corr = df_gene_all.dropna(subset=analysis_cols)
print(f"\nГенов после группировки: {len(df_gene_all)}, после dropna={len(df_gene_for_corr)}")

if len(df_gene_for_corr) < 5:
    print("Слишком мало данных для gene-level корреляции.")
else:
    cor_gene_all = df_gene_for_corr[analysis_cols].corr(method="pearson")
    print("\n[Gene-level] Общая корреляция:")
    display(cor_gene_all)


df_final shape = (22095, 10)


,Transcript stable ID,description_id,pred_borzoi,pred_ideal,pred_rmt_2,pred_rmt_4,pred_rmt_8,experiment,gene_id,TPM
0,ENST00000001008,ENCFF123KIW,4.712871,4.037295,0.570057,2.727709,3.736615,ENCSR892LBU,4478.0,22.30
1,ENST00000002596,ENCFF123KIWr,1.099219,4.281860,-2.881656,0.732307,1.674160,ENCSR892LBU,2587.0,12.38
2,ENST00000002829,ENCFF123KIW,3.475321,5.388129,1.192969,2.426938,0.736134,ENCSR892LBU,1617.0,12.69
3,ENST00000009180,ENCFF123KIW,6.688652,7.723541,2.320119,4.060136,4.146430,ENCSR892LBU,10278.0,133.66
4,ENST00000009589,ENCFF123KIWr,5.853962,8.866660,4.619201,4.862711,5.075033,ENCSR892LBU,8988.0,490.39



TPM statistics BEFORE log transformation:
Min: 0.0
Max: 9988.95
Mean: 9.9357922506183
Median: 0.29
Std: 101.09205449624558
Count of zeros: 5723

TPM statistics AFTER log transformation:
Min: -4.605170185988091
Max: 9.209235762119286
Mean: -1.0965484169716957
Median: -1.203972804325936
Std: 3.0377831837110976

Всего строк в df_final: 22095, после dropna=18195

[Transcript-level] Общая корреляция:


,pred_borzoi,pred_rmt_2,pred_rmt_4,pred_rmt_8,pred_ideal,TPM
pred_borzoi,1.000000,0.637651,0.766558,0.769407,0.775805,0.772642
pred_rmt_2,0.637651,1.000000,0.808983,0.762415,0.651431,0.636308
pred_rmt_4,0.766558,0.808983,1.000000,0.959157,0.772183,0.761828
pred_rmt_8,0.769407,0.762415,0.959157,1.000000,0.770695,0.764442
pred_ideal,0.775805,0.651431,0.772183,0.770695,1.000000,0.957162
TPM,0.772642,0.636308,0.761828,0.764442,0.957162,1.000000



Генов после группировки: 3624, после dropna=3624

[Gene-level] Общая корреляция:


,pred_borzoi,pred_rmt_2,pred_rmt_4,pred_rmt_8,pred_ideal,TPM
pred_borzoi,1.000000,0.650418,0.778773,0.781868,0.810392,0.804505
pred_rmt_2,0.650418,1.000000,0.814209,0.766975,0.684896,0.671260
pred_rmt_4,0.778773,0.814209,1.000000,0.964447,0.811660,0.801323
pred_rmt_8,0.781868,0.766975,0.964447,1.000000,0.810537,0.804418
pred_ideal,0.810392,0.684896,0.811660,0.810537,1.000000,0.965241
TPM,0.804505,0.671260,0.801323,0.804418,0.965241,1.000000
